# Copiar Datos desde GS a Google Drive
Ejecutar esta parte en Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()

project_id = 'clean-composite-253713'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://tfmuah2019/


In [13]:
bucket_name = 'tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-0003*'
dest = '/content/drive/My Drive/Colab Notebooks/data'

!gsutil -m cp -r  gs://{bucket_name} '/content/drive/My Drive/Colab Notebooks/data'

Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00030-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00038-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00033-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00031-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00039-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00037-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
Copying gs://tfmuah2019/data/20191026_080650_W100_A30_O20_S2.0/images/part-00034-c0836f3f-9c71-4f13-b3a2-af59c7feb756-c000.csv...
| [7/7 files][691.6 MiB/691.6 MiB] 100% Done  43.4 MiB/s ETA 00:00:00           
Operation

# Modificar Convertir Datos a Feather (train, validation, test)
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [1]:
import functions as func
import datasets
import pandas as pd
import numpy as np
import glob

Using TensorFlow backend.


In [2]:
files = glob.glob('data/*')
len(files)

33

In [3]:
datasets.copy_feather(files, grouping=3, reb=False)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [49:03<00:00, 283.17s/it]


# Rebalancear Dataset
Ejecutar esta parte en local desde la carpeta en la que esta "data/"

In [5]:
files = glob.glob('train_simple/*')
len(files)

57

In [8]:
suffix='simple_up'

In [11]:
from tqdm import tqdm
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == 1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather('train_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [12:00<00:00, 10.71s/it]


In [12]:
files = glob.glob('test_simple/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == 1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather('test_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [07:12<00:00,  7.46s/it]


In [13]:
files = glob.glob('validation_simple/*')
len(files)
i=0
for f in tqdm(files):
    df = func.toDF(f)
    df['LABEL'] = df['LABEL'].apply(lambda x: 1 if x == 1 else 0)
    df = func.rebalance(df)
    df.reset_index(drop=True).to_feather('validation_' + suffix + '/file_' + str(i) + ('.feather'))
    i=i+1

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [07:14<00:00,  7.35s/it]
